In [11]:
#definimos librerias para utilizar.

#debemos instalar Selenium "pip install selenium"
# Docs https://selenium-python.readthedocs.io
#debemos instalar el manejador del driver "pip install webdriver-manager"

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd 
import numpy as np

In [12]:
chrome_options = webdriver.ChromeOptions();
chrome_options.add_argument("--incognito");
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options);

In [14]:
#hacemos el request a la pagina.
#en este caso pre filtramos por la ciudad de Cali y tomamos la url de la pagina que consultamos
driver.get("https://www.metrocuadrado.com/apartamento-casa/venta/cali/?search=form");
time.sleep(3);

In [15]:
#identificamos la paginación, el boton de siguiente y traemos el total de registros
pagination = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[2]/li')
next_button = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[2]/li[12]')
total_items = int(driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[2]/li[11]/a').text)

In [16]:
def obtener_items():
    try:
        items = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[1]/li')
        print(f"--total items x pagina {len(items)}")
        return items
    except Exception as e:
        print(f'error al obtener los items')
        print(e)
        return bool([false]);

In [17]:
def traer_datos_item(item, url):
    try:
        
        try:
            s_url = requests.get(url);
            page = BeautifulSoup(s_url.text, 'lxml');
        except Exception as e:
            print('error al acceder al item', s_url)
            print(e)
            return None
        
        container = page.find('div', attrs={'class':'inner-container'}).contents
        
        datos_casa['Sector'] = container[3].contents[0].contents[1].p.get_text().split(',')[0].strip()
        datos_casa['Area'] = container[3].contents[0].contents[1].p.span.get_text().split(' ')[2]
        datos_casa['AreaUnidad'] = container[3].contents[0].contents[1].p.span.get_text().split(' ')[3]
        datos_casa['Precio'] = container[3].contents[0].contents[2].h3.get_text().replace('.', '').replace('$', '')
        datos_casa['Habitaciones'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[0].get_text()[0:1]
        datos_casa['Baños'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[1].get_text()[0:1]
        datos_casa['Estrato'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[1].h2.get_text()[0:1]

        datosPrincipales = container[3].contents[0].contents[7].div.contents;
        
        datosPrincipales

        for dato in datosPrincipales:
            datos_casa[dato.contents[0].get_text()] = dato.contents[1].get_text()

    except Exception as e:
        print(f'error al obtener los items')
        print(e)
 


In [18]:
def pasar_pagina():
    try:
        next_button.click()
        time.sleep(3)
    except Exception as e:
        print(f'error al pasar pagina')
        print(e)

In [19]:
#iniciamos el ciclo principal para la extracción de datos
try:
    print(f'**INICIANDO WEB SCRAPING A metrocuadrado.com**')
    datos = []
    ids = 1
    for numPage in range(total_items):
        print(f'-Obteniendo pagina {numPage + 1}')
        items = obtener_items();
        for item in items:
            item_url = item.find_element(By.XPATH,'.//div/ul/li[1]/a').get_attribute("href");
            datos_casa = {};
            datos_casa['url'] = item_url;
            datos_casa['id'] = ids;
            traer_datos_item(item, item_url)
            datos.append(datos_casa)        
            print(f'-dato {ids}')
            ids = ids + 1;
        pasar_pagina();   
    print(f'**FINAL DEL SCRAPING A metrocuadrado.com**')   
    pd.DataFrame(datos).to_csv('datos.csv')    
except Exception as e:
    print(f'Error recorrer la pagina')
    print(e)
    

        

**INICIANDO WEB SCRAPING A metrocuadrado.com**
-Obteniendo pagina 1
--total items x pagina 50
-dato 1
-dato 2
-dato 3
-dato 4
-dato 5
-dato 6
-dato 7
-dato 8
-dato 9
-dato 10
-dato 11
-dato 12
-dato 13
-dato 14
-dato 15
-dato 16
-dato 17
-dato 18
-dato 19
-dato 20
-dato 21
-dato 22
-dato 23
-dato 24
-dato 25
-dato 26
-dato 27
-dato 28
-dato 29
-dato 30
-dato 31
-dato 32
-dato 33
-dato 34
-dato 35
-dato 36
-dato 37
-dato 38
-dato 39
-dato 40
-dato 41
-dato 42
-dato 43
-dato 44
-dato 45
-dato 46
-dato 47
-dato 48
-dato 49
-dato 50
error al pasar pagina
Message: element click intercepted: Element <li class="item-icon-next page-item">...</li> is not clickable at point (966, 783). Other element would receive the click: <section class="Disclaimer__Disclamer-vmqhc8-0 jPbpVh disclamer-box">...</section>
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000104d03f48 chromedriver + 4226888
1   chromedriver                        0x0000000104cfc4f4 